In [1]:

from ib_insync import *
import pandas as pd
import time
from typing import Iterable, List

# Start/patch the event loop for notebooks (important!)
util.startLoop()

# ----------------- CONFIG (edit as you like) -----------------
HOST = "127.0.0.1"
PORT = 7497           # 7497 paper / 7496 live
CLIENT_ID = 111

UNDERLYING_SYMBOL = "AAPL"
EXCHANGE = "SMART"
CURRENCY = "USD"

INCLUDE_RIGHTS = ("C", "P")   # ("C",) for calls only; ("P",) for puts
# genericTicks for options: 100=Option Volume, 101=Open Interest, 104=Hist Vol, 106=Option IV
GENERIC_TICKS = "106"

BATCH_SIZE = 40               # contracts per batch (lower if you hit pacing)
BATCH_SLEEP_SEC = 2          # pause between batches
PER_BATCH_TIMEOUT_SEC = 20    # time to wait for greeks per batch
WRITE_CSV = False
CSV_PATH = "aapl_option_chain_with_greeks.csv"





In [2]:
import random
util.startLoop()  # once per kernel

def connect_ib(
    host="127.0.0.1",
    ports=(7497,) , #, 7496),           # paper, then live
    client_id_start=111,
    max_client_id_tries=10,
    timeout=5.0,
    bump_random_if_fail=True,
):
    """
    Connect to IB/TWS robustly:
    - tries ports in order
    - bumps clientId if 'already in use' or connection is closed immediately
    - ensures previous sockets are closed between attempts
    """
    last_err = None
    for port in ports:
        client_id = client_id_start
        for attempt in range(max_client_id_tries):
            ib = connect_ib()
            try:
                if ib.isConnected():
                    ib.disconnect()
                ib.connect(host, port, clientId=client_id, timeout=timeout)
                if ib.isConnected():
                    print(f"Connected to {host}:{port} with clientId {client_id}")
                    return ib
            except Exception as e:
                last_err = e
                # Common symptoms: peer closed connection, clientId in use, timeout
                try:
                    ib.disconnect()
                except Exception:
                    pass

                # next client id
                if bump_random_if_fail and attempt == 0:
                    client_id = random.randint(1000, 5000)
                else:
                    client_id += 1

                # brief backoff
                time.sleep(0.5)
        # try next port
    raise RuntimeError(f"Failed to connect after tries. Last error: {last_err!r}")


In [3]:
from ib_insync import *
import pandas as pd
import time
from typing import Iterable, List, Optional

util.startLoop()  # once per kernel

def fetch_option_chain_with_greeks(
    ib: IB,
    symbol: str = "AAPL",
    exchange: str = "SMART",
    currency: str = "USD",
    *,
    # Mode:
    # - "delayed_stream": uses delayed data + streaming (generic ticks OK, Greeks likely to populate)
    # - "snapshot_core":  snapshots only (no generic ticks allowed → Greeks often None)
    mode: str = "delayed_stream",
    # Chain narrowing:
    n_expiries: int = 2,
    strikes_each_side: int = 5,      # ±K around ATM
    rights: Iterable[str] = ("C","P"),
    # Market data behavior:
    include_volume_oi: bool = False, # if True (streaming only), adds 100/101; needs subs
    per_batch: int = 40,
    wait_sec: float = 10.0,          # how long to wait for ticks/Greeks per batch
    sleep_between: float = 5.0,      # pause between batches to respect pacing
    verbose: bool = True,
) -> pd.DataFrame:
    """
    Fetch a compact slice of the options chain with top-of-book and (where available) Greeks/IV.

    Modes:
      - delayed_stream: sets market data type 3 and streams with generic tick 106 (IV) by default, then cancels.
      - snapshot_core:  uses true snapshots (no generic ticks) — faster, but Greeks usually None.

    Tip: Start with small n_expiries / strikes_each_side, then scale up.
    """

    mode = mode.lower().strip()
    if mode not in ("delayed_stream", "snapshot_core"):
        raise ValueError("mode must be 'delayed_stream' or 'snapshot_core'")

    # ---------- Market data type ----------
    if mode == "delayed_stream":
        # Delayed data: avoids subscription errors for most accounts
        ib.reqMarketDataType(3)  # 1=RT, 2=Frozen, 3=Delayed, 4=Delayed-Frozen
        generic_ticks = ["106"]  # IV → helps modelGreeks populate
        if include_volume_oi:
            generic_ticks += ["100", "101"]  # option volume & open interest (subs often required)
        generic = ",".join(generic_ticks)
        use_snapshot = False
    else:
        # True snapshots: generic ticks are NOT allowed with snapshots
        ib.reqMarketDataType(3)
        generic = ""
        use_snapshot = True

    # ---------- Qualify underlying & discover chain ----------
    stock = Stock(symbol, exchange, currency)
    ib.qualifyContracts(stock)
    plist = ib.reqSecDefOptParams(symbol, "", stock.secType, stock.conId)
    chain = next((p for p in plist if p.exchange == exchange), plist[0])
    expiries = sorted(chain.expirations)[:max(0, n_expiries)]
    all_strikes = sorted(chain.strikes)
    if verbose:
        print(f"Found {len(chain.expirations)} expiries, {len(all_strikes)} strikes; using {len(expiries)} expiries.")

    # ---------- Get underlying (delayed) price for ATM centering ----------
    t_under = ib.reqMktData(stock, snapshot=True)
    # wait briefly for a price
    deadline = time.time() + 3.0
    und_px = None
    while time.time() < deadline:
        ib.waitOnUpdate(0.1)
        und_px = t_under.last or t_under.close or t_under.marketPrice()
        if und_px:
            break
    if und_px is None:
        # fallback: mid of listed strikes, or just use the median strike
        und_px = all_strikes[len(all_strikes)//2] if all_strikes else 0.0
    if verbose:
        print(f"Underlying (delayed) price guess: {und_px}")

    # ---------- Choose ±K strikes around ATM ----------
    if all_strikes:
        by_dist = sorted(all_strikes, key=lambda k: abs(k - und_px))
        core_strikes = sorted(by_dist[: (2*strikes_each_side + 1)])
    else:
        core_strikes = []
    if verbose:
        print(f"Using {len(core_strikes)} strikes around ATM: [{core_strikes[0] if core_strikes else 'N/A'} ... {core_strikes[-1] if core_strikes else 'N/A'}]")

    # ---------- Build & qualify option contracts ----------
    contracts: List[Option] = []
    for e in expiries:
        for k in core_strikes:
            for r in rights:
                contracts.append(
                    Option(
                        symbol=symbol,
                        lastTradeDateOrContractMonth=e,
                        strike=float(k),
                        right=r,
                        exchange=exchange,
                        currency=currency,
                        tradingClass=chain.tradingClass,
                        multiplier=chain.multiplier,
                    )
                )
    if not contracts:
        if verbose:
            print("No contracts chosen (check expiries/strikes selection).")
        return pd.DataFrame()

    contracts = ib.qualifyContracts(*contracts)

    # ---------- Request market data in batches ----------
    rows: List[dict] = []
    total = len(contracts)
    if verbose:
        print(f"Requesting {'SNAPSHOT' if use_snapshot else 'STREAMING'} data for {total} contracts...")

    def _collect(tk: Ticker) -> dict:
        c = tk.contract
        g = tk.modelGreeks or tk.lastGreeks or tk.bidGreeks or tk.askGreeks
        return {
            "underlying": symbol,
            "expiry": c.lastTradeDateOrContractMonth,
            "right": c.right,
            "strike": c.strike,
            "tradingClass": getattr(c, "tradingClass", None),
            "multiplier": getattr(c, "multiplier", None),
            "bid": tk.bid,
            "ask": tk.ask,
            "last": tk.last,
            "volume": tk.volume,
            "openInterest": getattr(tk, "openInterest", None),
            "iv": getattr(g, "impliedVol", None) if g else None,
            "delta": getattr(g, "delta", None) if g else None,
            "gamma": getattr(g, "gamma", None) if g else None,
            "theta": getattr(g, "theta", None) if g else None,
            "vega": getattr(g, "vega", None) if g else None,
            "optPrice_model": getattr(g, "optPrice", None) if g else None,
            "undPrice_model": getattr(g, "undPrice", None) if g else None,
            "mode": mode,
        }

    # batching
    for i in range(0, total, per_batch):
        batch = contracts[i : i + per_batch]
        if verbose:
            print(f"Batch {i//per_batch + 1}: {len(batch)} contracts")

        tickers = [ib.reqMktData(c, genericTickList=generic, snapshot=use_snapshot) for c in batch]

        # wait for data/Greeks to populate
        end = time.time() + wait_sec
        while time.time() < end:
            ib.waitOnUpdate(0.25)

        # collect rows
        rows.extend(_collect(tk) for tk in tickers)

        # cancel only for streaming mode
        if not use_snapshot:
            for tk in tickers:
                ib.cancelMktData(tk.contract)

        # pacing pause
        if (i + per_batch) < total and sleep_between > 0:
            if verbose:
                print(f"Sleeping {sleep_between}s...")
            ib.sleep(sleep_between)

    df = pd.DataFrame(rows)

    # ---------- sort & tidy ----------
    def _expiry_sort_key(s: Optional[str]) -> str:
        if not s:
            return "99999999"
        s = str(s)
        # pad YYYYMM → YYYYMM01; ensure length 8 for sorting
        return s if len(s) >= 8 else (s + "01").ljust(8, "0")

    if not df.empty:
        df = df.sort_values(
            by=["expiry", "strike", "right"],
            key=lambda col: col.map(_expiry_sort_key) if col.name == "expiry" else col,
        ).reset_index(drop=True)

    if verbose:
        print(f"Collected {len(df)} rows.")
    return df


In [4]:
# 2) Fast snapshots (usually NO Greeks):
df_snap = fetch_option_chain_with_greeks(
    ib=connect_ib(),
    symbol="AAPL",
    mode="snapshot_core",    # true snapshots; no generics allowed
    n_expiries=1,
    strikes_each_side=4,
)
df_snap.head()



: 

In [25]:
df.head()

,symbol,expiry,right,strike,tradingClass,multiplier,bid,ask,volume,iv,delta,gamma,theta,vega,optPrice_model,undPrice_model
0,AAPL,20250926,C,110.0,AAPL,100,145.8,146.30,0.0,2.841663,1.000195e+00,3.232252e-10,-6.175087e-02,2.681446e-09,1.465976e+02,256.314117
1,AAPL,20250926,P,110.0,AAPL,100,-1.0,0.01,0.0,2.841663,-2.120100e-09,3.232437e-10,-1.401052e-08,2.684099e-09,1.401052e-08,256.314117
2,AAPL,20250926,C,120.0,AAPL,100,135.4,136.30,0.0,2.525716,1.000195e+00,3.032549e-10,-6.291402e-02,1.167507e-09,1.365988e+02,256.314117
3,AAPL,20250926,P,120.0,AAPL,100,-1.0,0.01,0.0,2.525716,-1.764886e-09,3.032805e-10,-1.032092e-08,1.167843e-09,1.032092e-08,256.314117
4,AAPL,20250926,C,125.0,AAPL,100,130.4,131.65,2.0,2.370566,1.000195e+00,2.572166e-10,-6.349559e-02,1.190728e-09,1.315994e+02,256.314117


In [11]:
from ib_insync import *
import xml.etree.ElementTree as ET

util.startLoop()
ib = connect_ib()
ib.reqMarketDataType(3)  # 3=Delayed, switch to 1 if you have real-time

params_xml = ib.reqScannerParameters()   # XML catalog
root = ET.fromstring(params_xml)
codes = sorted({p.attrib['code'] for p in root.findall(".//scanTypeList/scanType")})
# Pick an options-focused scan code if present; fall back sensibly.
OPT_SCAN_CANDIDATES = [c for c in codes if "OPT" in c or "OPTION" in c]
OPT_SCAN_CANDIDATES[:10]


Connected to 127.0.0.1:7497 with clientId 111


[]

In [17]:
from ib_insync import ScannerSubscription

SCAN_CODE = next(s for s in OPT_SCAN_CANDIDATES if "OPT" in s or "OPTION" in s)  # pick what you prefer
sub = ScannerSubscription()
sub.instrument = "STK"              # underlyings are stocks
sub.locationCode = "STK.US.MAJOR"   # US majors; adjust if needed
sub.scanCode = SCAN_CODE
sub.numberOfRows = 100              # ask for top 100 (many scans allow >50)  # see note

# Request scan
scan_data = ib.reqScannerSubscription(sub, [])
symbols = [d.contractDetails.contract.symbol for d in scan_data]
len(symbols), symbols[:10]


StopIteration: 

In [18]:

def nearby_option_contracts(ib: IB, sym: str, exchange="SMART", currency="USD",
                            n_expiries=2, strikes_each_side=5, rights=("C","P")):
    stock = Stock(sym, exchange, currency)
    ib.qualifyContracts(stock)
    plist = ib.reqSecDefOptParams(sym, "", stock.secType, stock.conId)
    chain = next((p for p in plist if p.exchange == exchange), plist[0])
    expiries = sorted(chain.expirations)[:n_expiries]
    strikes = sorted(chain.strikes)

    # center around underlying price using delayed snapshot
    t = ib.reqMktData(stock, snapshot=True)
    ib.sleep(1.0)
    und_px = t.last or t.close or t.marketPrice()
    if und_px is None:
        return []

    # pick ±k strikes around ATM
    def pick_strikes():
        diffs = sorted(strikes, key=lambda k: abs(k - und_px))
        core = sorted(diffs[:(2*strikes_each_side+1)])
        return core
    core_strikes = pick_strikes()

    cs = []
    for e in expiries:
        for k in core_strikes:
            for r in rights:
                cs.append(Option(sym, e, float(k), r, exchange, currency, chain.tradingClass, chain.multiplier))
    return ib.qualifyContracts(*cs)

def snap_greeks(ib: IB, contracts, generic="100,101,104,106", per_batch=40, wait_sec=10, sleep_between=5):
    rows = []
    for chunk in (contracts[i:i+per_batch] for i in range(0, len(contracts), per_batch)):
        tickers = [ib.reqMktData(c, genericTickList=generic, snapshot=True) for c in chunk]
        deadline = time.time() + wait_sec
        while time.time() < deadline:
            ib.sleep(0.25)
        for tk in tickers:
            c = tk.contract
            g = tk.modelGreeks or tk.lastGreeks or tk.bidGreeks or tk.askGreeks
            rows.append({
                "symbol": c.symbol, "expiry": c.lastTradeDateOrContractMonth, "right": c.right, "strike": c.strike,
                "bid": tk.bid, "ask": tk.ask, "last": tk.last, "volume": tk.volume,
                "openInterest": getattr(tk, "openInterest", None),
                "iv": getattr(g, "impliedVol", None) if g else None,
                "delta": getattr(g, "delta", None) if g else None,
                "gamma": getattr(g, "gamma", None) if g else None,
                "theta": getattr(g, "theta", None) if g else None,
                "vega": getattr(g, "vega", None) if g else None,
            })
        if len(rows) % per_batch == 0:
            ib.sleep(sleep_between)
    return pd.DataFrame(rows)

# Example: pull first ~20 symbols’ nearby options to validate the flow
all_rows = []
for sym in symbols[:20]:
    cons = nearby_option_contracts(ib, sym, n_expiries=2, strikes_each_side=5)
    if cons:
        df_sym = snap_greeks(ib, cons)
        all_rows.append(df_sym.assign(underlying=sym))
df = pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()
df.head()

NameError: name 'symbols' is not defined